# OpenAI埋め込みのベクトルデータベースとしてQdrantを使用する

このノートブックでは、OpenAI埋め込みのベクトルデータベースとして**`Qdrant`**を使用する方法を段階的にガイドします。[Qdrant](https://qdrant.tech)は、Rustで書かれた高性能なベクトル検索データベースです。埋め込みを管理するためのRESTfulおよびgRPC APIを提供しています。アプリケーションとの統合を容易にする公式のPython [qdrant-client](https://github.com/qdrant/qdrant_client)があります。

このノートブックでは、以下のエンドツーエンドのプロセスを紹介します：
1. OpenAI APIで作成された事前計算済み埋め込みの使用
2. Qdrantのローカルインスタンスへの埋め込みの保存
3. OpenAI APIを使用した生テキストクエリの埋め込みへの変換
4. 作成されたコレクション内で最近傍検索を実行するためのQdrantの使用

### Qdrantとは

[Qdrant](https://qdrant.tech)は、ニューラル埋め込みをメタデータ（[payload](https://qdrant.tech/documentation/payload/)とも呼ばれる）と一緒に保存できるオープンソースのベクトルデータベースです。ペイロードは特定のポイントの追加属性を保持するためだけでなく、フィルタリングにも使用できます。[Qdrant](https://qdrant.tech)は、ベクトル検索フェーズに組み込まれた独自のフィルタリングメカニズムを提供しており、これにより非常に効率的になっています。

### デプロイメントオプション

[Qdrant](https://qdrant.tech)は、アプリケーションの対象負荷に応じて、さまざまな方法で起動できます：

- Dockerコンテナを使用してローカルまたはオンプレミスで
- [Helmチャート](https://github.com/qdrant/qdrant-helm)を使用してKubernetesクラスター上で
- [Qdrant Cloud](https://cloud.qdrant.io/)を使用して

### 統合

[Qdrant](https://qdrant.tech)は、RESTfulとgRPC APIの両方を提供しており、使用するプログラミング言語に関係なく統合が簡単です。ただし、最も人気のある言語向けの公式クライアントがいくつか利用可能で、Pythonを使用する場合は[Python Qdrantクライアントライブラリ](https://github.com/qdrant/qdrant_client)が最良の選択かもしれません。

In [1]:
! docker compose up -d

[+] Running 1/0
 ✔ Container qdrant-qdrant-1  Running                                      0.0s 


サーバーが正常に起動したかどうかを確認するために、簡単なcurlコマンドを実行して検証することができます：

In [2]:
! curl http://localhost:6333

{"title":"qdrant - vector search engine","version":"1.3.0"}

### 要件のインストール

このノートブックでは明らかに`openai`と`qdrant-client`パッケージが必要ですが、使用する他の追加ライブラリもいくつかあります。以下のコマンドでそれらすべてをインストールできます：

In [ ]:
! pip install openai qdrant-client pandas wget

### OpenAI APIキーの準備

OpenAI APIキーは、ドキュメントとクエリのベクトル化に使用されます。

OpenAI APIキーをお持ちでない場合は、[https://beta.openai.com/account/api-keys](https://beta.openai.com/account/api-keys)から取得できます。

キーを取得したら、以下のコマンドを実行して環境変数として`OPENAI_API_KEY`に追加してください：

In [4]:
! export OPENAI_API_KEY="your API key"

In [5]:
# Test that your OpenAI API key is correctly set as an environment variable
# Note. if you run this notebook locally, you will need to reload your terminal and the notebook for the env variables to be live.
import os

# Note. alternatively you can set a temporary env variable like this:
# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

if os.getenv("OPENAI_API_KEY") is not None:
    print("OPENAI_API_KEY is ready")
else:
    print("OPENAI_API_KEY environment variable not found")

OPENAI_API_KEY is ready


## Qdrantに接続する

公式のPythonライブラリを使用すると、実行中のQdrantサーバーインスタンスへの接続は簡単です：

In [6]:
import qdrant_client

client = qdrant_client.QdrantClient(
    host="localhost",
    prefer_grpc=True,
)

利用可能な任意のメソッドを実行して、接続をテストできます：

In [7]:
client.get_collections()


CollectionsResponse(collections=[])

## データの読み込み

このセクションでは、このセッション以前に準備されたデータを読み込みます。これにより、あなた自身のクレジットを使ってWikipedia記事の埋め込みを再計算する必要がありません。

In [8]:
import wget

embeddings_url = "https://cdn.openai.com/API/examples/data/vector_database_wikipedia_articles_embedded.zip"

# The file is ~700 MB so this will take some time
wget.download(embeddings_url)

100% [......................................................................] 698933052 / 698933052

'vector_database_wikipedia_articles_embedded (9).zip'

ダウンロードしたファイルを展開する必要があります：

In [9]:
import zipfile

with zipfile.ZipFile("vector_database_wikipedia_articles_embedded.zip","r") as zip_ref:
    zip_ref.extractall("../data")

そして最後に、提供されたCSVファイルからそれを読み込むことができます：

In [10]:
import pandas as pd

from ast import literal_eval

article_df = pd.read_csv('../data/vector_database_wikipedia_articles_embedded.csv')
# Read vectors from strings back into a list
article_df["title_vector"] = article_df.title_vector.apply(literal_eval)
article_df["content_vector"] = article_df.content_vector.apply(literal_eval)
article_df.head()

,id,url,title,text,title_vector,content_vector,vector_id
0,1,https://simple.wikipedia.org/wiki/April,April,April is the fourth month of the year in the J...,"[0.001009464613161981, -0.020700545981526375, ...","[-0.011253940872848034, -0.013491976074874401,...",0
1,2,https://simple.wikipedia.org/wiki/August,August,August (Aug.) is the eighth month of the year ...,"[0.0009286514250561595, 0.000820168002974242, ...","[0.0003609954728744924, 0.007262262050062418, ...",1
2,6,https://simple.wikipedia.org/wiki/Art,Art,Art is a creative activity that expresses imag...,"[0.003393713850528002, 0.0061537534929811954, ...","[-0.004959689453244209, 0.015772193670272827, ...",2
3,8,https://simple.wikipedia.org/wiki/A,A,A or a is the first letter of the English alph...,"[0.0153952119871974, -0.013759135268628597, 0....","[0.024894846603274345, -0.022186409682035446, ...",3
4,9,https://simple.wikipedia.org/wiki/Air,Air,Air refers to the Earth's atmosphere. Air is a...,"[0.02224554680287838, -0.02044147066771984, -0...","[0.021524671465158463, 0.018522677943110466, -...",4


## インデックスデータ

Qdrantは**コレクション**にデータを保存し、各オブジェクトは少なくとも1つのベクトルで記述され、**ペイロード**と呼ばれる追加のメタデータを含む場合があります。私たちのコレクションは**Articles**と呼ばれ、各オブジェクトは**title**と**content**の両方のベクトルで記述されます。Qdrantでは事前にスキーマを設定する必要がないため、簡単な設定のみでコレクションにポイントを自由に配置できます。

まずコレクションを作成し、その後事前に計算された埋め込みでそれを埋めていきます。

In [11]:
from qdrant_client.http import models as rest

vector_size = len(article_df["content_vector"][0])

client.create_collection(
    collection_name="Articles",
    vectors_config={
        "title": rest.VectorParams(
            distance=rest.Distance.COSINE,
            size=vector_size,
        ),
        "content": rest.VectorParams(
            distance=rest.Distance.COSINE,
            size=vector_size,
        ),
    }
)

True

In [12]:
client.upsert(
    collection_name="Articles",
    points=[
        rest.PointStruct(
            id=k,
            vector={
                "title": v["title_vector"],
                "content": v["content_vector"],
            },
            payload=v.to_dict(),
        )
        for k, v in article_df.iterrows()
    ],
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [13]:
# Check the collection size to make sure all the points have been stored
client.count(collection_name="Articles")

CountResult(count=25000)

## データの検索

データがQdrantに格納されたら、最も近いベクトルを求めてコレクションにクエリを開始します。タイトルベースの検索からコンテンツベースの検索に切り替えるために、追加パラメータ`vector_name`を提供することができます。事前計算された埋め込みは`text-embedding-ada-002` OpenAIモデルで作成されたため、検索時にも同じモデルを使用する必要があります。

In [14]:
from openai import OpenAI

openai_client = OpenAI()

def query_qdrant(query, collection_name, vector_name="title", top_k=20):
    # Creates embedding vector from user query
    embedded_query = openai_client.embeddings.create(
        input=query,
        model="text-embedding-ada-002",
    ).data[0].embedding

    query_results = client.search(
        collection_name=collection_name,
        query_vector=(
            vector_name, embedded_query
        ),
        limit=top_k,
    )

    return query_results

In [15]:
query_results = query_qdrant("modern art in Europe", "Articles")
for i, article in enumerate(query_results):
    print(f"{i + 1}. {article.payload['title']} (Score: {round(article.score, 3)})")

1. Museum of Modern Art (Score: 0.875)
2. Western Europe (Score: 0.867)
3. Renaissance art (Score: 0.864)
4. Pop art (Score: 0.86)
5. Northern Europe (Score: 0.855)
6. Hellenistic art (Score: 0.853)
7. Modernist literature (Score: 0.847)
8. Art film (Score: 0.843)
9. Central Europe (Score: 0.843)
10. European (Score: 0.841)
11. Art (Score: 0.841)
12. Byzantine art (Score: 0.841)
13. Postmodernism (Score: 0.84)
14. Eastern Europe (Score: 0.839)
15. Cubism (Score: 0.839)
16. Europe (Score: 0.839)
17. Impressionism (Score: 0.838)
18. Bauhaus (Score: 0.838)
19. Surrealism (Score: 0.837)
20. Expressionism (Score: 0.837)


In [16]:
# This time we'll query using content vector
query_results = query_qdrant("Famous battles in Scottish history", "Articles", "content")
for i, article in enumerate(query_results):
    print(f"{i + 1}. {article.payload['title']} (Score: {round(article.score, 3)})")

1. Battle of Bannockburn (Score: 0.869)
2. Wars of Scottish Independence (Score: 0.861)
3. 1651 (Score: 0.852)
4. First War of Scottish Independence (Score: 0.85)
5. Robert I of Scotland (Score: 0.846)
6. 841 (Score: 0.844)
7. 1716 (Score: 0.844)
8. 1314 (Score: 0.837)
9. 1263 (Score: 0.836)
10. William Wallace (Score: 0.835)
11. Stirling (Score: 0.831)
12. 1306 (Score: 0.831)
13. 1746 (Score: 0.83)
14. 1040s (Score: 0.828)
15. 1106 (Score: 0.827)
16. 1304 (Score: 0.826)
17. David II of Scotland (Score: 0.825)
18. Braveheart (Score: 0.824)
19. 1124 (Score: 0.824)
20. Second War of Scottish Independence (Score: 0.823)
